 # Competición titanic con keras


 ## importar librerias y datos

In [123]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

item_categories = pd.read_csv('data/item_categories.csv')
items = pd.read_csv('data/items.csv')
sales_train = pd.read_csv('data/sales_train.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')
shops = pd.read_csv('data/shops.csv')
test = pd.read_csv('data/test.csv')

datasets = {
    'item_categories': item_categories,
    'items': items,
    'sales_train': sales_train,
    'sample_submission': sample_submission,
    'shops': shops,
    'test': test
}

In [124]:
for key,value in datasets.items():
    print(f"{key} => {value.columns.values}")

item_categories => ['item_category_name' 'item_category_id']
items => ['item_name' 'item_id' 'item_category_id']
sales_train => ['date' 'date_block_num' 'shop_id' 'item_id' 'item_price' 'item_cnt_day']
sample_submission => ['ID' 'item_cnt_month']
shops => ['shop_name' 'shop_id']
test => ['ID' 'shop_id' 'item_id']


# Dataset a meses

In [125]:
x_train = sales_train.groupby(['date_block_num','shop_id','item_id']).sum()['item_cnt_day'].reset_index()
x_train.tail()

,date_block_num,shop_id,item_id,item_cnt_day
1609119,33,59,22087,6.0
1609120,33,59,22088,2.0
1609121,33,59,22091,1.0
1609122,33,59,22100,1.0
1609123,33,59,22102,1.0


 ## Ingeniera de datos

In [126]:
y_train = x_train.item_cnt_day
x_train.drop(['item_cnt_day'], axis=1, inplace=True)


# data[:891].Survived

# # Borrar columnas innesesarias.
# features_to_remove = ['PassengerId', 'Ticket', 'Cabin',  'Survived', 'Fare', ]
# data = data.drop(features_to_remove , axis=1)
# #  Reemplazar datos categóricos por de sexo 0s y  por 1s.
# data['Sex'] = LabelEncoder().fit_transform(data['Sex'])

Verificamos si hay valores nulos.

Pclass        0
Name          0
Sex           0
Age         263
SibSp         0
Parch         0
Embarked      2
dtype: int64

 ### Generación de nuevos atributos

 A partir de ``SibSp`` y ``Parch`` generamos y reemplazamos lo anterior con un nuevo atributo para verificar si el pasajero arrivó solo o con algún pariente: ``IsAlone``.

In [5]:
# data['IsAlone'] = 0
# data.loc[ (data['SibSp'] + data['Parch']) == 0, 'IsAlone'] = 1 
# data.drop(['SibSp', 'Parch'], axis=1, inplace=True)

 A partir de Los prefijos de los nombres del los pasajeros, se genera un nuevo atributo que representa el prefijo de los nombres: ``Title``.


In [6]:
# Extraer prefijo del nombre de las personas.
# data['Title'] = data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

# Reemplazar los titulos raros con uno mas abarcativo: 'Rare'.
# data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
# data['Title'] = data['Title'].replace(['Mlle','Ms'], 'Miss')
# data['Title'] = data['Title'].replace('Mme', 'Mrs')
# data['Title'] = LabelEncoder().fit_transform(data['Title'])
# data.drop(['Name'], axis=1, inplace=True)


 Rellenamos ``Age`` faltantes según la relación con ``Sex`` y ``Pclass``.

In [7]:
# guess_ages = np.zeros((2,3))

# for i in range(0, 2): # itera con => 0,1 para sexo
#     for j in range(0, 3):  # itera con => 1,2,3 para pclass
#         guess_df = data[(data['Sex'] == i) & (data['Pclass'] == j+1)]['Age'].dropna()

#         # Convierte el numero decimal al .5 mas cercano
#         guess_ages[i,j] = round( guess_df.mean() *2 ) / 2

#         data.loc[ (data.Age.isnull()) & (data.Sex == i) & (data.Pclass == j+1),'Age'] = guess_ages[i,j]

# data["AgeCat"]= pd.cut(data["Age"], bins=[0, 15, 30, 45, 60, max(data["Age"]+1)], labels=[1, 2, 3, 4, 5])
# data.drop(['Age'], axis=1, inplace=True)


 rellamos ``Embarked`` con el valor mas comun

In [8]:
# data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

 Convertimos columna de datos categóricos a variables dummys.

In [9]:
# data = pd.get_dummies(data, columns=['Pclass', 'Embarked', 'Title'], prefix=['Pclass','Embarked', 'Title'])#, drop_first=True)

 Split y visualización de ``data``.

In [10]:
# X_train = data[:891]
# test = data[891:]

In [127]:
x_train.tail()

,date_block_num,shop_id,item_id
1609119,33,59,22087
1609120,33,59,22088
1609121,33,59,22091
1609122,33,59,22100
1609123,33,59,22102


 ## Definición de modelo

In [128]:
def build_model(lr):

  model = keras.Sequential([
    keras.layers.LSTM(32, input_shape=[3,1]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation='relu') 
  ])

  model.compile(loss='mse',
                optimizer=Adam(learning_rate=lr),
                metrics=['accuracy'])

  return model

 ## Entrenamiento del modelo


### Instaciacion del modelo.

In [129]:
keras.utils.set_random_seed(0)
model = build_model(0.1)

### entrenamiento normal

In [ ]:
history = model.fit(
    x_train,
    y_train,
    verbose=2, epochs=25, batch_size=x_train.shape[0])

 ## Evaluar con data 100% accuracy
 para evitar tener que estar subiendo constantemente la submission a kaggle, lo comparamos con el resultado final de la competición que se encuentra en un repo en github.

In [14]:
y_test = pd.read_csv('data/submission_100accuracy.csv')['Survived']

model.evaluate(test, y_test)

14/14 [==============================] - 0s 2ms/step - loss: 0.5080 - accuracy: 0.7871


[0.5079693794250488, 0.7870813608169556]

 ## Predicciones

In [188]:
preds = model.predict(test)
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0

14/14 [==============================] - 0s 2ms/step


 ## Exportar predicciones

In [189]:
prediction = pd.DataFrame({'PassengerId': test_data.PassengerId.values, 'Survived': preds.ravel().astype('int64')})
prediction.to_csv('data/results.csv', index=False)

In [190]:
prediction.head(5)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
